In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    temperature=0.1,
)

# memory_key속성의 "chat_history" key가 template안의 chat_history와 같아야 한다.
# 메모리는 template안의 대화내용을 저장할 공간을 찾아 자동으로 입력한다.
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",
)

template = """
    당신은 도움이 되는 인공지능으로 사람과 대화하고 있습니다.

    {chat_history}
    Human: {question}
    You:
"""
# off the shelf
chain = LLMChain(
    memory=memory,
    llm=llm,
    verbose=True,  # chain의 prompt 로그를 확인 할 수 있다.
    prompt=PromptTemplate.from_template(template),
)

chain.predict(question="나의 이름은 앤더슨 입니다.")



> Entering new LLMChain chain...
Prompt after formatting:

    당신은 도움이 되는 인공지능으로 사람과 대화하고 있습니다.

    
    Human: 나의 이름은 앤더슨 입니다.
    You:


> Finished chain.


'안녕하세요, 앤더슨님. 만나서 반가워요. 어떻게 도와드릴까요?'

In [2]:
chain.predict(question="나는 서울에 살고있습니다.")



> Entering new LLMChain chain...
Prompt after formatting:

    당신은 도움이 되는 인공지능으로 사람과 대화하고 있습니다.

    Human: 나의 이름은 앤더슨 입니다.
AI: 안녕하세요, 앤더슨님. 만나서 반가워요. 어떻게 도와드릴까요?
    Human: 나는 서울에 살고있습니다.
    You:


> Finished chain.


'AI: 서울은 정말 멋진 도시죠! 어떤 도움이 필요하신가요? 위치 정보나 관광 정보를 알려드릴까요?'

In [3]:
chain.predict(question="나는 김치를 좋아합니다.")



> Entering new LLMChain chain...
Prompt after formatting:

    당신은 도움이 되는 인공지능으로 사람과 대화하고 있습니다.

    System: The human introduces themselves as Anderson. The AI greets Anderson in Korean and asks how it can help. Anderson mentions that they live in Seoul.
AI: AI: 서울은 정말 멋진 도시죠! 어떤 도움이 필요하신가요? 위치 정보나 관광 정보를 알려드릴까요?
    Human: 나는 김치를 좋아합니다.
    You:


> Finished chain.


'AI: 그렇군요! 김치는 한국의 전통 음식 중 하나로 매우 맛있는 음식이에요. 서울에는 많은 맛있는 김치를 즐길 수 있는 식당들이 있어요. 궁금하신 점이 있거나 추천을 원하시면 언제든지 물어봐주세요!'

In [4]:
chain.predict(question="나의 이름은 무엇입니까?")



> Entering new LLMChain chain...
Prompt after formatting:

    당신은 도움이 되는 인공지능으로 사람과 대화하고 있습니다.

    System: The human introduces themselves as Anderson and mentions they live in Seoul. The AI greets Anderson in Korean and offers to provide location or tourist information. Anderson expresses their love for kimchi. The AI acknowledges kimchi as a delicious traditional Korean food and offers recommendations for restaurants in Seoul.
    Human: 나의 이름은 무엇입니까?
    You:


> Finished chain.


'안녕하세요, Anderson 씨! 당신의 이름은 Anderson이고, 저는 여러분을 돕기 위해 여기 있습니다. 어떤 도움이 필요하신가요?'